<a href="https://colab.research.google.com/github/jjhonxpx/confeitaria_projeto01/blob/main/SDV/SDV_confeitaria.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Instala a biblioteca SDV
!pip install sdv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.6/196.6 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.4/139.4 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 62.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.7/52.7 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.3/74.3 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.3/198.3 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 36.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.7/85.7 kB 4.0 MB/s eta 0:00:00


In [2]:
# Importa a biblioteca pandas, usada para manipulação de dados tabulares
import pandas as pd

# Importa o sintetizador GaussianCopula do SDV, modelo estatístico para gerar dados sintéticos a partir de uma única tabela
from sdv.single_table import GaussianCopulaSynthesizer

# Importa a classe para criar ou detectar automaticamente o metadado da sua tabela
from sdv.metadata import SingleTableMetadata

# Importa a classe FixedCombinations, que impede o SDV de criar combinações entre colunas que não existem nos dados originais
from sdv.cag import FixedCombinations

In [3]:
confeitaria_df = pd.read_excel("/content/relatorio_item_vendas_limpo.xlsx")
confeitaria_df.head().round(2)

,Produto,Categoria,Quantidade Vendida,Faturamento Total,Preço Médio,Custo Médio,Custo Total,Lucro,Margem
0,10 SALGADINHOS (5 FRANGO + 5 CARNE DE SOL),SALGADOS,202,3434.0,17.00,0,0,3434.0,1.0
1,2 EMPADAS - 130G (FRANGO E CARNE DE SOL),SALGADOS,12,328.0,27.33,0,0,328.0,1.0
2,EMPADÃO - CARNE DE SOL - 1KG (8 FATIAS),SALGADOS,27,2835.0,105.00,0,0,2835.0,1.0
3,EMPADÃO - FRANGO - 500G (5 FATIAS),SALGADOS,137,6165.0,45.00,0,0,6165.0,1.0
4,1 EMPADA - 130G (FRANGO),SALGADOS,463,6482.0,14.00,0,0,6482.0,1.0


In [4]:
# Colunas que devem manter apenas as combinações existentes nos dados reais
constraint = FixedCombinations(
    column_names=['Produto', 'Categoria']
)

In [5]:
# Gera o metadado automaticamente
metadata = SingleTableMetadata()
metadata.detect_from_dataframe(confeitaria_df)
print('Metadado detectado:', metadata.to_dict()) # Veja o tipo de dado de cada coluna

Metadado detectado: {'columns': {'Produto': {'sdtype': 'categorical'}, 'Categoria': {'sdtype': 'categorical'}, 'Quantidade Vendida': {'sdtype': 'numerical'}, 'Faturamento Total': {'sdtype': 'numerical'}, 'Preço Médio': {'sdtype': 'numerical'}, 'Custo Médio': {'sdtype': 'categorical'}, 'Custo Total': {'sdtype': 'categorical'}, 'Lucro': {'sdtype': 'numerical'}, 'Margem': {'sdtype': 'numerical'}}, 'METADATA_SPEC_VERSION': 'SINGLE_TABLE_V1'}


In [6]:
# Cria o sintetizador
synthesizer = GaussianCopulaSynthesizer(metadata)

# Adiciona restrições ao sintetizador usando add_constraints
synthesizer.add_constraints([constraint])  # Lista de constraints

/usr/local/lib/python3.12/dist-packages/sdv/single_table/base.py:168: FutureWarning: The 'SingleTableMetadata' is deprecated. Please use the new 'Metadata' class for synthesizers.
  warnings.warn(DEPRECATION_MSG, FutureWarning)
/usr/local/lib/python3.12/dist-packages/sdv/single_table/base.py:134: UserWarning: We strongly recommend saving the metadata using 'save_to_json' for replicability in future SDV versions.
  warnings.warn(


In [7]:
# Ajustar o modelo ao seu conjunto real
synthesizer.fit(confeitaria_df)

In [8]:
# Gerar novas linhas sintéticas
sintetico = synthesizer.sample(num_rows=30)
print(sintetico.head())

                                   Produto  \
0                      PUDIM - ZERO (100G)   
1  TORTA COOKIE COM NUTELLA DA CASA (115G)   
2               SANDUÍCHE - OURO DO SERTÃO   
3                           TORTA DE LIMÃO   
4                   TORTA DE BANANA (120G)   

                                      Categoria  Quantidade Vendida  \
0                           FATIAS E SOBREMESAS                 175   
1                           FATIAS E SOBREMESAS                   3   
2  PÃES E SANDUÍCHES - SEM GLÚTEN E SEM LACTOSE                  54   
3                           FATIAS E SOBREMESAS                 219   
4                           FATIAS E SOBREMESAS                 328   

   Faturamento Total  Preço Médio  Custo Médio  Custo Total    Lucro    Margem  
0             2948.2     9.896260            0            0   2276.7  0.958461  
1              155.9    20.217116            0            0    163.8  0.983059  
2             3658.7     8.661247            0       

In [9]:
pd.options.display.max_rows = None  # Permite mostrar todas as linhas

sintetico.round(2)

,Produto,Categoria,Quantidade Vendida,Faturamento Total,Preço Médio,Custo Médio,Custo Total,Lucro,Margem
0,PUDIM - ZERO (100G),FATIAS E SOBREMESAS,175,2948.2,9.90,0,0,2276.7,0.96
1,TORTA COOKIE COM NUTELLA DA CASA (115G),FATIAS E SOBREMESAS,3,155.9,20.22,0,0,163.8,0.98
2,SANDUÍCHE - OURO DO SERTÃO,PÃES E SANDUÍCHES - SEM GLÚTEN E SEM LACTOSE,54,3658.7,8.66,0,0,2970.5,1.00
3,TORTA DE LIMÃO,FATIAS E SOBREMESAS,219,14354.3,127.40,0,0,13514.4,1.00
4,TORTA DE BANANA (120G),FATIAS E SOBREMESAS,328,1681.0,5.94,0,0,2034.7,1.00
5,"NAKED ""TE AMO"" (3/4 PESSOAS)",NAKEDS CAKE & TORTAS,60,1154.1,80.16,0,0,1249.3,1.00
6,NAKED PISTACHE & NINHO + GEL. MORANGO(10 PESSOAS),NAKEDS CAKE & TORTAS,6,453.2,22.10,0,0,397.2,1.00
7,SUCO DE MORANGO (250ML),OUTRAS BEBIDAS,3,202.5,109.61,0,0,479.0,0.97
8,QUICHE - TOMATE SECO & MANJERICÃO,FÉRIAS - PIQUENIQUE - TEMPO DE APROVEITAR,2,90.9,50.65,0,0,126.3,1.00
9,BANANA DULCE (4 FATIAS) - ZERO AÇÚCAR,PERFEITOS PARA CAFÉ - INCLUSIVOS,2,181.2,283.72,0,0,104.2,0.97


In [12]:
# Pós-processamento: calcula faturamento, custo, lucro e margem a partir dos dados sintéticos
sintetico['Faturamento Total'] = sintetico['Quantidade Vendida'] * sintetico['Preço Médio']
sintetico['Custo Total'] = sintetico['Quantidade Vendida'] * sintetico['Custo Médio']
sintetico['Lucro'] = sintetico['Faturamento Total'] - sintetico['Custo Total']
sintetico['Margem'] = sintetico['Lucro'] / sintetico['Faturamento Total']

sintetico.round(2)

,Produto,Categoria,Quantidade Vendida,Faturamento Total,Preço Médio,Custo Médio,Custo Total,Lucro,Margem
0,PUDIM - ZERO (100G),FATIAS E SOBREMESAS,175,1731.85,9.90,0,0,1731.85,1.0
1,TORTA COOKIE COM NUTELLA DA CASA (115G),FATIAS E SOBREMESAS,3,60.65,20.22,0,0,60.65,1.0
2,SANDUÍCHE - OURO DO SERTÃO,PÃES E SANDUÍCHES - SEM GLÚTEN E SEM LACTOSE,54,467.71,8.66,0,0,467.71,1.0
3,TORTA DE LIMÃO,FATIAS E SOBREMESAS,219,27901.63,127.40,0,0,27901.63,1.0
4,TORTA DE BANANA (120G),FATIAS E SOBREMESAS,328,1949.51,5.94,0,0,1949.51,1.0
5,"NAKED ""TE AMO"" (3/4 PESSOAS)",NAKEDS CAKE & TORTAS,60,4809.50,80.16,0,0,4809.50,1.0
6,NAKED PISTACHE & NINHO + GEL. MORANGO(10 PESSOAS),NAKEDS CAKE & TORTAS,6,132.62,22.10,0,0,132.62,1.0
7,SUCO DE MORANGO (250ML),OUTRAS BEBIDAS,3,328.82,109.61,0,0,328.82,1.0
8,QUICHE - TOMATE SECO & MANJERICÃO,FÉRIAS - PIQUENIQUE - TEMPO DE APROVEITAR,2,101.29,50.65,0,0,101.29,1.0
9,BANANA DULCE (4 FATIAS) - ZERO AÇÚCAR,PERFEITOS PARA CAFÉ - INCLUSIVOS,2,567.45,283.72,0,0,567.45,1.0


In [11]:
# Exportar planilha sintética em excel e csv
sintetico.to_csv('dados_sinteticos_sdv.csv', index=False)
sintetico.to_excel('dados_sinteticos_sdv.xlsx', index=False)